In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm, trange
import os
from typing import Mapping, Union, Optional, Callable, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, Subset, DataLoader, SubsetRandomSampler, TensorDataset, ConcatDataset
import warnings
warnings.filterwarnings(action='ignore')

## Dataset and data distribution

In [ ]:
SAMPLING_RATE = 32000 # Hertz
INPUT_LENGTH = 5      # seconds
SPEC_SHAPE = (48,128) # spectrogram shape
FMIN=500              # Hz (~min frequency for birds)
FMAX=12500            # Hz (~max frequency for birds)

In [ ]:
DATA_MEAN = 0.3674
DATA_STD = 0.1928

META_MEAN = [24.472804615395624, -79.96082831218337, 162.2867799090244, 619.9162929032668]
META_STD = [22.10848093451316, 38.37072607349947, 88.93816236402917, 266.60113126777134]

## Metadata

In [ ]:
# Load metadata file
metadata_df = pd.read_csv('../input/birdclef-2021/train_metadata.csv')
all_species = metadata_df['primary_label'].unique()
meta_df = metadata_df[["filename","primary_label","latitude","longitude","date","time","rating"]]  # retain
# TIME OF DAY NOT PRESENT ON TEST DATA: USELESS
meta_df

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

# express DATE in days (remove year)
def date_transform(date):
    return int(date.rsplit('-')[2]) + max(0, int(date.rsplit('-')[1])-1)*30
meta_df['date'] = meta_df['date'].apply(date_transform)

# express TIME in minutes
def time_transform(time):
   # exclude '?', 'xx', 'xx:xx', 'night', 'am'
    if not(any(map(str.isdigit, time))):     # if no numbers inside
        return np.random.randint(0,1440)     # method1: in this way we will not increase a specific time slot
    else:
        Q = 0
        time = time.lower()                  # AM,PM->am,pm
        if 'pm' in time:
            Q = 1
        time = time.replace('pm','')
        time = time.replace('am','')
        time = time.rsplit(':')
        minutes = 0
        if len(time)>1:
            minutes = int(time[1])
        result = (int(time[0]) + Q*12)*60 + minutes
        return( result%1440 )                # there are also '12:30pm', '12:15pm', '24:15', etc
meta_df['time'] = meta_df['time'].apply(time_transform)
meta_df

## The model

In [ ]:
# (ReLU BEFORE AND AFTER ADDITION) +dropout 
class ResBlock1(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.drop2d = nn.Dropout2d(p=0.1)

    def forward(self, input):
        shortcut = self.shortcut(input)
        
        input = nn.ReLU()(self.bn1(self.conv1(input)))          # pool substituted with stride
        input = self.drop2d(input)
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)

In [ ]:
# ResNet34 first version
class ResNet34(nn.Module):
    def __init__(self, in_channels, resblock, outputs=397):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = nn.Sequential(
            resblock(64, 64, downsample=False),
            resblock(64, 64, downsample=False),
            resblock(64, 64, downsample=False)
        )

        self.layer2 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False),
            resblock(128, 128, downsample=False),
            resblock(128, 128, downsample=False)
        )

        self.layer3 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False),
            resblock(256, 256, downsample=False),
            resblock(256, 256, downsample=False),
            resblock(256, 256, downsample=False),
            resblock(256, 256, downsample=False)
        )


        self.layer4 = nn.Sequential(
            resblock(256, 512, downsample=True),
            resblock(512, 512, downsample=False),
            resblock(512, 512, downsample=False),
        )

        self.meta_rep = 64
        self.gap = torch.nn.AdaptiveAvgPool2d(1)        
        self.fc = torch.nn.Linear(512 + self.meta_rep*3, outputs)
        self.drop = nn.Dropout(p=0.4)

    def forward(self, input):
        
        D = input[0]  # data
        M = input[1]  # meta
        Lat = M[:,0].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        Lon = M[:,1].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        Date = M[:,2].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        
        D = self.layer0(D)
        D = self.layer1(D)
        D = self.layer2(D)
        D = self.layer3(D)
        D = self.layer4(D)
        D = self.gap(D)                # [64, feature_maps, h, w] -> [64, feature_maps, 1]

        D = D.view(D.shape[0], -1)     # [64, feature_maps, 1]    ->  [64, feature_maps]
        D = self.drop(D)   
        DM = torch.cat((D, Lat, Lon, Date),1)    # data & meta
        
        DM = self.fc(DM)

        return DM

In [ ]:
class ResBottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        self.downsample = downsample
        self.conv1 = nn.Conv2d(in_channels, out_channels//4, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(out_channels//4, out_channels//4, kernel_size=3, stride=2 if downsample else 1, padding=1)
        self.conv3 = nn.Conv2d(out_channels//4, out_channels, kernel_size=1, stride=1)
        self.shortcut = nn.Sequential()
        
        if self.downsample or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2 if self.downsample else 1),
                nn.BatchNorm2d(out_channels)
            )

        self.bn1 = nn.BatchNorm2d(out_channels//4)
        self.bn2 = nn.BatchNorm2d(out_channels//4)
        self.bn3 = nn.BatchNorm2d(out_channels)
        
        self.drop2d = nn.Dropout2d(p=0.2)      # batchnorm alone leads to overfitting

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = self.drop2d(input)
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = self.drop2d(input)
        input = nn.ReLU()(self.bn3(self.conv3(input)))
        input = input + shortcut
        return nn.ReLU()(input)

In [ ]:
class ResNet(nn.Module):
    def __init__(self, in_channels, resblock, repeat, useBottleneck=False, outputs=397):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        if useBottleneck:
            filters = [64, 256, 512, 1024, 2048]
            self.meta_rep = 128
        else:
            filters = [64, 64, 128, 256, 512]
            self.meta_rep = 32

        self.layer1 = nn.Sequential()
        self.layer1.add_module('conv2_1', resblock(filters[0], filters[1], downsample=False))
        for i in range(1, repeat[0]):
                self.layer1.add_module('conv2_%d'%(i+1,), resblock(filters[1], filters[1], downsample=False))

        self.layer2 = nn.Sequential()
        self.layer2.add_module('conv3_1', resblock(filters[1], filters[2], downsample=True))
        for i in range(1, repeat[1]):
                self.layer2.add_module('conv3_%d' % (i+1,), resblock(filters[2], filters[2], downsample=False))

        self.layer3 = nn.Sequential()
        self.layer3.add_module('conv4_1', resblock(filters[2], filters[3], downsample=True))
        for i in range(1, repeat[2]):
            self.layer3.add_module('conv2_%d' % (i+1,), resblock(filters[3], filters[3], downsample=False))

        self.layer4 = nn.Sequential()
        self.layer4.add_module('conv5_1', resblock(filters[3], filters[4], downsample=True))
        for i in range(1, repeat[3]):
            self.layer4.add_module('conv3_%d'%(i+1,), resblock(filters[4], filters[4], downsample=False))

        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = torch.nn.Linear(filters[4] + self.meta_rep*3, outputs)
        self.drop = nn.Dropout(p=0.4)

    def forward(self, input):
        
        D = input[0]  # data
        M = input[1]  # meta
        Lat = M[:,0].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        Lon = M[:,1].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        Date = M[:,2].unsqueeze(0).transpose(0,1).repeat(1,self.meta_rep)
        
        D = self.layer0(D)
        D = self.layer1(D)
        D = self.layer2(D)
        D = self.layer3(D)
        D = self.layer4(D)
        D = self.gap(D)                # [64, feature_maps, h, w] -> [64, feature_maps, 1]
        
        D = D.view(D.shape[0], -1)     # [64, feature_maps, 1, 1]    ->  [64, feature_maps]
        D = self.drop(D)
        
        DM = torch.cat((D, Lat, Lon, Date),1)    # data & meta
        
        DM = self.fc(DM)

        return DM

In [ ]:
def count_parameters(model: torch.nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# Define the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}') 

# resnet50
resnet50 = ResNet(1, ResBottleneckBlock, [3, 4, 6, 3], useBottleneck=True, outputs=397)
print('Model initialization')

## Soundscapes

In [ ]:
# load model on cpu

model = ResNet34(in_channels = 1, resblock = ResBlock1)
model.to(device)

# model = resnet50
# model.to(device)

print(f'Number of parameters: {count_parameters(model)}')
model.load_state_dict(torch.load('../input/models/ResNet34.pt', map_location=torch.device('cpu')))


In [ ]:
def get_TEST_landscape_spectrograms(audio_path, output_dir):
    
    sig, rate = librosa.load(audio_path, sr=SAMPLING_RATE, offset=None)
    
    lunp_samples = int(INPUT_LENGTH*SAMPLING_RATE)   # number of time samples in a 5s piece of signal (5s*32000Hz)
    
    # split signal into five second lumps
    sig_splits = []
    for i in range(0, len(sig), lunp_samples):
        split = sig[i:i + lunp_samples]

        # End of signal
        if len(split) < lunp_samples:
            break
        sig_splits.append(split)
    
    # extract mel spectrograms
    split_count = 0
    samples = []
    for lump in sig_splits:
        
        HOP_LENGTH = int((INPUT_LENGTH*SAMPLING_RATE)/(SPEC_SHAPE[1]-1))
        mel_spec = librosa.feature.melspectrogram(y=lump,
                                                 sr=SAMPLING_RATE,
                                                 n_fft=1024,
                                                 hop_length=HOP_LENGTH,
                                                 n_mels=SPEC_SHAPE[0],
                                                 fmin=FMIN,
                                                 fmax=FMAX)
        
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    
        # Normalize
        mel_spec -= mel_spec.min()
        mel_spec /= mel_spec.max()
        
        # Save as image file
        # THIS TIME IN A SINGLE FOLDER (NO LABELS):
        spec_5s_name = audio_path.rsplit(os.sep, 1)[-1].rsplit('_',1)[0] + '_' + str((split_count+1)*5) + '.png'
        specs_names.append(spec_5s_name)
        save_path = os.path.join(output_dir, spec_5s_name)
        
        im = Image.fromarray(mel_spec * 255.0).convert("L") # color -> greyscale (mode “L”): L = R * 299/1000 + G * 587/1000 + B * 114/1000
        im.save(save_path)
        
        samples.append(save_path)
        split_count += 1
            
    filename = audio_path.rsplit(os.sep, 1)[-1].rsplit('.', 1)[0]
    location = filename.rsplit('_',2)[1]
    meta_lat.append(geo_map[filename.rsplit('_',2)[1]][0])
    meta_lon.append(geo_map[filename.rsplit('_',2)[1]][1])
    date = filename.rsplit('_',1)[1][0:4] + '-' + filename.rsplit('_',1)[1][4:6] + '-' + filename.rsplit('_',1)[1][6:8]
    meta_date.append(date_transform(date))
    return samples

In [ ]:
import re
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
# TEST SOUNDSCAPES
import librosa

def list_files(path):
    return [f for f in os.listdir(path) if f.rsplit('.', 1)[-1] in ['ogg']]      # return name list audio.ogg

# if commit: test folder not populated; if rerun by kaggle: test folder populated
input_dir = '../input/birdclef-2021/test_soundscapes'
audio_list = list_files(input_dir)
if len(audio_list) == 0:
    input_dir = '../input/birdclef-2021/train_soundscapes'
    audio_list = list_files(input_dir)

output_dir = './TEST_SOUNDSCAPES_MEL_DATASET'
os.makedirs(output_dir)
TEST_SPECS = []
geo_map = {'COL': [5.57,-75.85], 'COR':[10.12, -84.51], 'SNE':[38.49,-119.95], 'SSW':[42.47,-76.45]}
meta_lat = []
meta_lon = []
meta_date = []

specs_names = []

for filename in audio_list:
    audio_file_path = os.path.join(input_dir, filename)
    TEST_SPECS += get_TEST_landscape_spectrograms(audio_file_path, output_dir)

print('SUCCESSFULLY EXTRACTED {} SPECTROGRAMS'.format(len(TEST_SPECS)))  # 2400

In [ ]:
# import shutil
# shutil.rmtree('./TEST_SOUNDSCAPES_MEL_DATASET')

In [ ]:
# META
meta_sound_df = pd.DataFrame(list(zip(meta_lat, meta_lon, meta_date)), columns =['latitude', 'longitude', 'date'])
print(meta_sound_df)

# NORMALIZE 
for idx, col in enumerate(['latitude', 'longitude', 'date']):
    meta_sound_df[col] = ( meta_sound_df[col] - META_MEAN[idx] ) / META_STD[idx]

a = meta_sound_df.columns    
# REPEAT: repeat each row in meta_sound_df 600s/5s times:
meta_sound_df = pd.DataFrame(np.repeat(meta_sound_df.values, 120, axis=0))
meta_sound_df.columns = a
meta_sound_df.shape

In [ ]:
meta_sound_tensor = torch.tensor([meta_sound_df['latitude'].values, meta_sound_df['longitude'].values, meta_sound_df['date'].values]).float()
meta_sound_tensor = torch.transpose(meta_sound_tensor, 0, 1)
meta_sound_tensor, meta_sound_tensor.shape

In [ ]:
transform_inference =  transforms.Compose([transforms.Grayscale(), 
                                           transforms.ToTensor(), 
                                           transforms.Normalize(mean=DATA_MEAN, std=DATA_STD)])
def image_loader(image_path):
    """load image, returns cuda tensor"""
    image = Image.open(image_path)
    image = transform_inference(image)
    image = image.unsqueeze(0)  # adds dimension 0 of size 1 (batch dim) to obtain 1x1x48x128
    if device.type == 'cuda':
        return image.cuda()
    else:
        return image

In [ ]:
row_list = natural_sort(specs_names)
COL_list = []
COR_list = []
SNE_list = []
SSW_list = []
for row in row_list:
    if 'COL' in row:
        COL_list.append(row)
    if 'COR' in row:
        COR_list.append(row)
    if 'SNE' in row:
        SNE_list.append(row)
    if 'SSW' in row:
        SSW_list.append(row)
        
ordered_list = COL_list + COR_list + SNE_list + SSW_list      # sort submission.csv according to this list

In [ ]:
# birds_as_nocalls10 = list(['balori', 'bkcchi', 'bobfly1', 'chswar', 'comyel', 'eastow', 'gockin', 'reevir1', 'rewbla', 'rucwar', 'sonspa'])
# birds_as_nocalls15 = list(['bkcchi', 'bobfly1', 'comyel', 'reevir1', 'rewbla', 'rucwar', 'sonspa'])

In [ ]:
# list in a dataframe pair of birds frequently heard together
from itertools import combinations
truth = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv')

groups = truth[truth['birds'].str.split().str.len() > 1]
groups['birds'] = groups['birds'].apply(lambda x: list(combinations(x.split(' '), 2)))
pairs = groups.explode('birds', ignore_index=True)
pairs_count = pd.DataFrame(np.stack(np.unique(pairs['birds'], return_counts = True), axis = 1),
                                   columns = ['birds', 'occurrences'])
common_pairs = pairs_count[pairs_count['occurrences']>10]    # maybe there are few pairs in hidden test set
common_pairs['birds'] = [' '.join(map(str, l)) for l in common_pairs['birds']]
common_pairs = common_pairs.sort_values(by=['occurrences'], ascending=False)
common_pairs

In [ ]:
# Predict MULTILABEL
test_soundscapes_dir =  './TEST_SOUNDSCAPES_MEL_DATASET'
from torch.nn import AvgPool2d
from collections import deque

Col_averager = AvgPool2d(kernel_size =(6,128*4))                     # 20 seconds
Patch_averager = AvgPool2d(kernel_size =(6,4))
last_saved = 0          # saved last_birds number (between these, accept lower probabilities for species1)
X = 10                  # number of highest score guesses to check for birds related to species1 (multilabel)
multilabel = False      # multi-birds label is inaccurate

# Store the results
data = {'row_id': [], 'birds': [], 'score': []}
with torch.no_grad():
    cnt = 0
    last_birds = deque(maxlen=last_saved)
    for idx, spec_name in enumerate(ordered_list):
        if idx % 120 == 0:                               # new 10 minutes audio (120*5s)
            last_birds = deque(maxlen=last_saved)
            # print('audio n. ', int(idx/120))
        path = os.path.join(test_soundscapes_dir, spec_name)
        
        spec = image_loader(path)
        meta = meta_sound_tensor[idx].unsqueeze(0)
        spec, meta = spec.to(device), meta.to(device)
        model.eval()
        
        # repeat 2/4/6 times to reach 10/20/30 seconds
        spec = torch.cat((spec, spec, spec, spec),-1)               # 20 seconds
        prediction = nn.functional.softmax(model([spec, meta]))[0]
        prediction = torch.Tensor.cpu(prediction)
        
        # store 1st, 2nd and first X guesses (the latter to be used in case of birds frequently heard together)
        idx2,idx1 = np.argpartition(prediction, -2)[-2:]            # indices of the 2 highest scores (max on the right)
        bestX_idx = np.argpartition(prediction, -X)[-X:]            # indices of the X highest scores
        species1, species2 = all_species[idx1], all_species[idx2]
        score1, score2 = prediction[idx1], prediction[idx2]         # 2 highest scores
        bestX = list(all_species[bestX_idx])                        # X highest scores
        bestX.remove(species1)                                      # (species1 removed)
        
        data['row_id'].append(spec_name.rsplit('.', 1)[0])
        
        # filter2: separate birdcalls | nocalls
        AvgCol = Col_averager(spec)            # 1x1x8x1
        AvgPatches = Patch_averager(spec)      # 1x1x8x128
        diff = AvgPatches - AvgCol             # 1x1x8x128
        diff = diff.view(diff.shape[0], -1)    # 1x(8*128)
        birdcall = np.any(np.absolute(np.array(torch.Tensor.cpu(diff)))>0.63, axis=1)
        
        bird2_flag = 0
        if birdcall and (species1 != 'grhowl') and (score1>0.33 or (last_birds.count(species1)>=3 and score1>0.15)):# or (species1 in birds_as_nocalls10 and score1>0.3):
            last_birds.append(species1)
            if len(common_pairs):
                has_rel_birds = common_pairs['birds'].str.contains(species1)                   # if 'common_pairs' rows contain species1
                if any(has_rel_birds):
                    rel_pairs = pd.DataFrame(common_pairs.loc[has_rel_birds])                  # rows containing species1
                    bestX_pattern = '|'.join(bestX)
                    rel_pairs['in_bestX'] = rel_pairs['birds'].str.contains(bestX_pattern)     # rows True if bird2 in row is in bestX
                    rel_pairs_inbestX = rel_pairs.loc[rel_pairs['in_bestX']]['birds'].to_list()   # list of pairs with bird2 in bestX
                    if rel_pairs_inbestX:
                        rel_bird = rel_pairs_inbestX[0].rsplit(' ')            # keep first pair, split it
                        rel_bird.remove(species1)                           # bird related to species1
                        rel_bird = str(rel_bird)
                        bird2_flag = 1
                        species2 = rel_bird

            if score1 < 0.40:     # add 'nocall' label anyway
                if multilabel and bird2_flag:
                    data['birds'].append({species1, species2, 'nocall'})
                    cnt +=2
                else:
                    data['birds'].append({species1, 'nocall'})
                    cnt += 1
            else:
                if multilabel and bird2_flag:
                    data['birds'].append({species1, species2})
                    cnt +=2
                else:
                    data['birds'].append({species1})
                    cnt += 1
        else:
            data['birds'].append({'nocall'})
        # Add the confidence score
        data['score'].append( str(score1.item()) + ' ' + str(score2.item()) )

print('SOUNDSCAPE ANALYSIS DONE. FOUND {} BIRDS.'.format(cnt))

In [ ]:
import shutil
shutil.rmtree('./TEST_SOUNDSCAPES_MEL_DATASET')

In [ ]:
# from sets to strings
prediction_dict = {'row_id' : [], 'birds' : []}
for row in data['row_id']:
    prediction_dict['row_id'].append(row)

for row in data['birds']:
    prediction_dict['birds'].append(' '.join(row))

In [ ]:
# Create submission file
submission = pd.DataFrame(prediction_dict, columns = ['row_id', 'birds'])
submission.to_csv("submission.csv", index=False)

In [ ]:
submission

In [ ]:
results = pd.DataFrame(data, columns = ['row_id', 'birds', 'score'])       # new dataframe
results.rename(columns = {'birds':'prediction'}, inplace = True)
results = pd.merge(truth, results, on='row_id')                            # merge with true labels

# Let's look at some entries
results[1076:1086]

In [ ]:
all_species_nocall = np.append('nocall', all_species)

from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes = all_species_nocall)

y_prediction = list(results['prediction'])
y_pred_matrix = mlb.fit_transform(y_prediction)

# true birds list to list of sets
y_truth = []
for element in list(results['birds']):
    y_truth.append(set(element.split()))
y_true_matrix = mlb.fit_transform(y_truth)

f1 = f1_score(y_true_matrix, y_pred_matrix, average='micro')
print('F1 = %.3f' %(f1))

# f1_398 = f1_score(y_true_matrix, y_pred_matrix, average=None)            # f1 for each class
# print('\n', 'F1_classes:\n', f1_398)